In [852]:
import numpy as np
import pandas as pd
import nltk

In [853]:
data=pd.read_csv("data.csv")

In [854]:
data.head()

,Rating,Title,Date,Verified Purchase,Body,Helpful Votes
0,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,Not really sure why I decided to try out the S...,01-Apr-18,Yes,I am a die hard Apple person. All my desktop c...,632.0
2,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,It works!,22-Apr-18,Yes,MetroPCS initially had issues getting it added...,88.0
4,NaN,NaN,NaN,NaN,NaN,NaN


In [855]:
data.dropna(inplace=True)

In [856]:
data['title_body']=data['Title']+ ' ' + data['Body']

In [857]:
data.reset_index(inplace=True)
data.drop(['index','Title','Body','Date','Verified Purchase','Helpful Votes'],axis=1,inplace=True)

In [858]:
data.head()

,Rating,title_body
0,5.0,Not really sure why I decided to try out the S...
1,5.0,It works! MetroPCS initially had issues gettin...
2,5.0,Switched from iPhone I switched to the s9 afte...
3,4.0,Good phone except for Bixby and Samsung bloatw...
4,5.0,Love it! Upgraded from the s6. Love it! Finger...


In [859]:
inp,out = np.array(data['title_body']),np.array(data['Rating'])

In [860]:
import string
from nltk.corpus import stopwords
stop = stopwords.words('english') + ['phone','iphone','x','apple','samsung']
for i in range(len(inp)):
    new_doc=''
    for j in range(len(inp[i])):
        if not inp[i][j] in string.punctuation:
            new_doc+=(inp[i][j])
        else:
            new_doc+=' '
    new_doc.strip()
    inp[i]=new_doc.strip()

In [861]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [862]:
lem=WordNetLemmatizer()

def get_simple_tag(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def NotNumber(w):
    for ch in w:
        if(ch>='0' and ch<='9'):
            return False
    return True
    
def Clean_Words(doc):
    doc=word_tokenize(doc)
    doc = [word for word in doc if word.lower() not in stop and NotNumber(word)==True]
    pos = pos_tag(doc)
    cleaned_words = [lem.lemmatize(w,pos=get_simple_tag(t)).lower() for w,t in pos]
    string=""
    for w in cleaned_words:
        string+=w
        string+=' '
    return string.strip()

In [863]:
clean_in=np.array([Clean_Words(doc) for doc in inp])
sentiment = ['negative','negative','negative','neutral','positive','positive']
clean_out=np.array([sentiment[int(rating)] for rating in out])

In [864]:
np.array([len(doc.split()) for doc in clean_in]).sum(),np.array([len(doc.split()) for doc in clean_in]).mean()

(12579, 31.213399503722083)

In [865]:
np.savetxt('Word Clouds/S9_neg_clean_reviews.csv',np.array(['text']+[clean_in[i] for i in range(len(clean_in)) if clean_out[i]=='negative']),fmt='%s',delimiter=',')
np.savetxt('Word Clouds/S9_neut_clean_reviews.csv',np.array(['text']+[clean_in[i] for i in range(len(clean_in)) if clean_out[i]=='neutral']),fmt='%s',delimiter=',')
np.savetxt('Word Clouds/S9_pos_clean_reviews.csv',np.array(['text']+[clean_in[i] for i in range(len(clean_in)) if clean_out[i]=='positive']),fmt='%s',delimiter=',')

In [866]:
from sklearn.model_selection import train_test_split

In [867]:
x_train,x_test,y_train,y_test=train_test_split(clean_in,clean_out)

In [868]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [869]:
vec=TfidfVectorizer(max_features=3000,max_df=0.7,min_df=0.1,ngram_range=(1,2))

In [870]:
vec.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=3000, min_df=0.1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [871]:
x_train=vec.transform(x_train)
x_test=vec.transform(x_test)

In [872]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [873]:
clf1 = MultinomialNB(alpha=0.1)
clf1.fit(x_train,y_train)
clf1.score(x_test,y_test)

0.7821782178217822

In [874]:
y_pred = clf1.predict(x_test)

In [875]:
from sklearn.metrics import classification_report

In [876]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf1.score(x_test,y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.29      0.11      0.16        18
    neutral       0.00      0.00      0.00         2
   positive       0.82      0.95      0.88        81

avg / total       0.71      0.78      0.73       101

Accuracy =  78.21782178217822 %


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [877]:
clf2=BernoulliNB()
clf2.fit(x_train.todense(),y_train)
clf2.score(x_test.todense(),y_test)

0.693069306930693

In [878]:
y_pred = clf2.predict(x_test.todense())

In [879]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf2.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.33      0.44      0.38        18
    neutral       0.12      0.50      0.20         2
   positive       0.88      0.75      0.81        81

avg / total       0.77      0.69      0.72       101

Accuracy =  69.3069306930693 %


In [880]:
clf3 = LogisticRegression()
clf3.fit(x_train,y_train)
clf3.score(x_test,y_test)

0.7821782178217822

In [881]:
y_pred = clf3.predict(x_test.todense())

In [882]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf3.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.20      0.06      0.09        18
    neutral       0.00      0.00      0.00         2
   positive       0.81      0.96      0.88        81

avg / total       0.69      0.78      0.72       101

Accuracy =  78.21782178217822 %


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [883]:
clf4 = RandomForestClassifier()
clf4.fit(x_train,y_train)
clf4.score(x_test,y_test)

0.7722772277227723

In [884]:
y_pred=clf4.predict(x_test)

In [885]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf4.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.44      0.39      0.41        18
    neutral       0.00      0.00      0.00         2
   positive       0.86      0.88      0.87        81

avg / total       0.76      0.77      0.77       101

Accuracy =  77.22772277227723 %


In [886]:
clf5 = SVC()
clf5.fit(x_train,y_train)
clf5.score(x_test,y_test)

0.801980198019802

In [887]:
y_pred=clf5.predict(x_test)

In [888]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf5.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.00      0.00      0.00        18
    neutral       0.00      0.00      0.00         2
   positive       0.80      1.00      0.89        81

avg / total       0.64      0.80      0.71       101

Accuracy =  80.19801980198021 %


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
